In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from three_tank_data.data_gen import ThreeTankDataGenerator
import pandas as pd
import constants as const
import pysindy as ps
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
ttdg = ThreeTankDataGenerator(number_initial_states=100)

In [ ]:
z, z_dot, time, uid_initial_state = ttdg.generate_z_space_data()

In [ ]:
df = pd.DataFrame(z, columns=const.Z_COL_NAMES)
df[const.Z_DOT_COL_NAMES] = z_dot
df[const.TIME_COL_NAME] = time
df[const.UID_INITIAL_STATE_COL_NAME] = uid_initial_state

In [ ]:
df

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

def _create_ts_plot(idx):
    df_plot = df[df[const.UID_INITIAL_STATE_COL_NAME] == idx]
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    # signal 1
    for col, name in zip(const.Z_COL_NAMES, ['h1(t)', 'h2(t)', 'h3(t)']):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1),
            row=1, col=1)

    for col, name in zip(const.Z_DOT_COL_NAMES, ['h1_dot(t)', 'h2_dot(t)', 'h3_dot(t)']):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1),
            row=2, col=1)

    fig.update_xaxes(title_text='time')
    fig.update_yaxes(title_text='x', row=1)
    fig.update_yaxes(title_text='x_dot', row=2)
    fig.update_layout(title_text="Latent neuron activations vs. hidden states", showlegend=True)
    fig.show()

interact(_create_ts_plot, idx=list(range(ttdg.number_initial_states)))

In [ ]:
library_functions = [
    lambda x : np.exp(x),
    lambda x : 1./x,
    lambda x : x,
    lambda x : np.sin(x),
    lambda x : np.cos(x),
    lambda x,y : np.sin(x+y),
    lambda x,y : np.cos(x+y),
    lambda x,y : np.sign(x-y)*np.sqrt(np.abs(x-y)),
    lambda x: x**2,
    lambda x: np.sqrt(x),
    lambda x,y: x*y,
    lambda x,y: x**2*y,
    lambda x,y: x*y**2
]
library_function_names = [
    lambda x : 'exp(' + x + ')',
    lambda x : '1/' + x,
    lambda x : x,
    lambda x : f'sin({x})',
    lambda x : f'cos({x})',
    lambda x,y : 'sin(' + x + '+' + y + ')',
    lambda x,y : f'cos({x}+{y})',
    lambda x,y : 'sign('+x+'-'+y+')*sqrt('+x+' - '+y+')',
    lambda x: '{'+x+'}^2',
    lambda x: f'sqrt({x})',
    lambda x,y: f'{x}*{y}',
    lambda x,y: f'{x}^2*{y}',
    lambda x,y: f'{x}*{y}^2'
]
feature_library = ps.CustomLibrary(
    library_functions=library_functions, function_names=library_function_names
)

In [ ]:
optimizer = ps.STLSQ(threshold=.1)
model = ps.SINDy(
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=["h1", "h2", "h3"]
)
model.fit(df[const.Z_COL_NAMES].values, x_dot=df[const.Z_DOT_COL_NAMES].values, t=0.1)
model.print()

In [ ]:
feature_library.get_feature_names()

In [ ]:
x, x_dot, time, uid_initial_state = ttdg.generate_x_space_data()

In [ ]:
x.shape

In [ ]:
x_dot.shape

In [ ]:
time.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import plotly.express as px
import numpy as np
indexes = [0, 25, 50, 75, 100]
indexes = [i+101*0 for i in indexes]
img_sequences = [1-x[i,:].reshape(100,100) for i in indexes]
fig = px.imshow(np.array(img_sequences),
                facet_col=0,
                binary_string=True,
                labels={'facet_col':'sigma'}
               )
for i, index in enumerate(indexes):
    fig.layout.annotations[i]['text'] = f'Image of fill-leves at timestep {index}'
fig.show()